<a href="https://colab.research.google.com/github/akm2208/Aayushi-First-Files/blob/main/posch_tutorial_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installations
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 18.9 MB/s eta 0:00:00


In [2]:
# imports
import os
import rasterio
from rasterio.warp import reproject
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [3]:
# drive file structure - mount and change to directory of the notebook
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
os.chdir('/content/gdrive/My Drive/Tutorial_Jan25_Satellite_ML/notebooks')

Mounted at /content/gdrive


Define function to read in raster data.

In [4]:
def read_one_raster(path):
    # file is expected in bil format (or any rasterio-compatible format)

    with rasterio.open(path) as src:
        arr = src.read(1)
        crs = src.crs
        affine = src.transform

    return arr, crs, affine # return the other things in order to post-verify
                         # that everything happened properly

Read in one temperature raster:

In [ ]:
folder = '../data/temperature/prism_raw'
YYYYMM = '201803'
file = f'PRISM_tmax_stable_4kmM3_{YYYYMM}_bil.bil'
path = f'{folder}/{file}'

In [ ]:
arr, crs, affine = read_one_raster(path)

Explore what these look like - arr, crs, affine

In [ ]:
arr

In [ ]:
plt.imshow(arr)

In [ ]:
crs

In [ ]:
affine

Read in all the temperature data that we care about. Put them in nice lists.

In [ ]:
# we will have temperature data from years 2018 thru 2022, and for each year use months 03 thru 08.

In [ ]:
def get_monthly_temp_paths(years=['2018','2019','2020','2021','2022'],
                          months=['03','04','05','06','07','08'],
                          folder='../data/temperature/prism_raw'):
    YYYYMMs = []
    for year in years:
        for month in months:
            YYYYMMs.append(year + month)
    files = [f'PRISM_tmax_stable_4kmM3_{YYYYMM}_bil.bil' for YYYYMM in YYYYMMs]
    paths = [f'{folder}/{file}' for file in files]

    return YYYYMMs, paths # list of all paths

In [ ]:
def get_data_from_paths(paths):
    arrs = []
    crss = []
    affines = []
    for path in paths:
        arr, crs, affine = read_one_raster(path)
        arrs.append(arr)
        crss.append(crs)
        affines.append(affine)

    return arrs, crss, affines

In [ ]:
YYYYMMs, prism_paths = get_monthly_temp_paths(years=['2018','2019','2020','2021','2022'],
                          months=['03','04','05','06','07','08'],
                          folder='../data/temperature/prism_raw')

prism_arrs, prism_crss, prism_affines = get_data_from_paths(prism_paths)

Check that all the CRSs and all the Affines are the same.

In [ ]:
def confirm_affines_same(affines):
    if len(set(affines)) == 1:
        print('Yes all affines same')
    else:
        print('No not same, checking what the affines are...')
        print(set(affines))
        print('If the above are within a tiny error you\'re good,')
        print('otherwise, go back and match your affines.')

In [ ]:
def confirm_crss_same(crss):
    if len(set(crss)) == 1:
        print('Yes all CRSs same')
    else:
        print('No not same, checking what the CRSs are...')
        print(set(crss))
        print('If the above are actually the same CRS you\'re good,')
        print('otherwise, go back and match your CRSs.')

In [ ]:
confirm_affines_same(prism_affines)

In [ ]:
confirm_crss_same(prism_crss)

Read in one surface reflectance raster.

In [ ]:
folder = '../data/surface_reflectance/hls_raw'
file = 'HLS.S30.T10SFH.2018285T185321.v2.0/HLS.S30.T10SFH.2018285T185321.v2.0.B03.tif'
# B03 means Green color band. (B02 is blue, B04 is red, etc)
# 2018285 means Year 2018 Day 285. So this is in October.
path = f'{folder}/{file}'

In [ ]:
arr, crs, affine = read_one_raster(path)

In [ ]:
arr

In [ ]:
plt.imshow(arr)

In [ ]:
crs

In [ ]:
affine

Read in all the surface reflectance data we care about.

In [ ]:
hls_folder = '../data/surface_reflectance/hls_raw'
hls_files = ['HLS.S30.T10SFH.2018285T185321.v2.0/HLS.S30.T10SFH.2018285T185321.v2.0.B03.tif',
         'HLS.S30.T10SFH.2019285T185319.v2.0/HLS.S30.T10SFH.2019285T185319.v2.0.B03.tif',
         'HLS.S30.T10SFH.2020285T185321.v2.0/HLS.S30.T10SFH.2020285T185321.v2.0.B03.tif',
         'HLS.S30.T10SFH.2021284T185319.v2.0/HLS.S30.T10SFH.2021284T185319.v2.0.B03.tif',
         'HLS.S30.T10SFH.2022284T185321.v2.0/HLS.S30.T10SFH.2022284T185321.v2.0.B03.tif']
hls_paths = [f'{hls_folder}/{file}' for file in hls_files]

In [ ]:
sr_arrs, sr_crss, sr_affines = get_data_from_paths(hls_paths)

Check that all CRSs and all Affines are the same.

In [ ]:
confirm_crss_same(sr_crss)

In [ ]:
confirm_affines_same(sr_affines)

Reproject and aggregate the surface reflectance data to match the temperature data.

In [ ]:
# reproject surface reflectance data

end = rasterio.open(prism_paths[0])

reprojected_hls_arrs = []
reprojected_hls_affines = []

for hls_path in hls_paths:
    print(f'Working on HLS data {hls_path}...')
    with rasterio.open(hls_path) as start:
        array, affine = rasterio.warp.reproject(source=start.read(1), # this is the starting array
                  destination=end.read(1), # the array of the end goal projection
                  src_transform=start.transform, # this is the transform corresponding to start array
                  src_crs=start.crs, # this is the crs corresponding to start array
                  dst_transform=end.transform, # this is the transform corresponding to end array
                  dst_crs=end.crs, # this is the crs corresponding to end array
                  resampling=rasterio.enums.Resampling.average # aggregate temperature using average
                  )

    reprojected_hls_arrs.append(array)
    reprojected_hls_affines.append(affine)

print('Done.')

Look at one of the reprojected arrays.

In [ ]:
# look at one
reprojected_hls_arrs[0]

In [ ]:
plt.imshow(reprojected_hls_arrs[0])

In [ ]:
plt.imshow(reprojected_hls_arrs[0][250:300,75:125])
# Exercise: take a look at all 5 hls arrays (just change the index from 0 to 1 to 2 etc)

In [ ]:
# Bonus exercise: check that all the reprojected affines are the same as the PRISM temperature affines!

Create a numpy array in machine learning format.

In [ ]:
# First, create a Boolean mask of the Sacramento area (the tile where we have HLS data)
sacramento = reprojected_hls_arrs[0] != 0.  # when we reprojected, the rest of USA was set to 0

In [ ]:
# note how many grid cells we have in our study area
sum(sum(sacramento))

In [ ]:
# create ML features for year 2018
# 6 features (temperature in March, April, ..., August)
features_2018 = []
for arr in prism_arrs[:6]:
    feature = arr[sacramento] # grab only the grid cells in our study area
    features_2018.append(feature)

# stack them into a numpy array - this is X for our 2018 data.
X_2018 = np.column_stack(features_2018)

In [ ]:
# look at it to see that it worked
X_2018

In [ ]:
X_2018.shape  # check that there are 719 observations (rows) and 6 features (columns)

In [ ]:
# create ML features X for years 2019, 2020, 2021, 2022
features_2019 = []
for arr in prism_arrs[6:12]:
    feature = arr[sacramento] # grab only the grid cells in our study area
    features_2019.append(feature)
X_2019 = np.column_stack(features_2019)

features_2020 = []
for arr in prism_arrs[12:18]:
    feature = arr[sacramento] # grab only the grid cells in our study area
    features_2020.append(feature)
X_2020 = np.column_stack(features_2020)

features_2021 = []
for arr in prism_arrs[18:24]:
    feature = arr[sacramento] # grab only the grid cells in our study area
    features_2021.append(feature)
X_2021 = np.column_stack(features_2021)

features_2022 = []
for arr in prism_arrs[24:30]:
    feature = arr[sacramento] # grab only the grid cells in our study area
    features_2022.append(feature)
X_2022 = np.column_stack(features_2022)

In [ ]:
# create ML targets y for years 2018-2022
y_2018 = reprojected_hls_arrs[0][sacramento]
y_2019 = reprojected_hls_arrs[1][sacramento]
y_2020 = reprojected_hls_arrs[2][sacramento]
y_2021 = reprojected_hls_arrs[3][sacramento]
y_2022 = reprojected_hls_arrs[4][sacramento]

In [ ]:
# stack years 2018-2021 into X_train and y_train
# need to stack verticvally (row-wise) - end result has 6 features a couple thousand observations
X_train = np.row_stack([X_2018,X_2019,X_2020,X_2021])
y_train = np.concatenate([y_2018,y_2019,y_2020,y_2021])

# confirm we ended up with the correct shape
print('X_train has shape', X_train.shape)
print('y_train has shape', y_train.shape)

In [ ]:
# use 2022 data for our validation set
X_val = X_2022
y_val = y_2022

# confirm we ended up with the correct shape
print('X_val has shape', X_val.shape)
print('y_val has shape', y_val.shape)

Hooray! Now we have a nice formatted dataset!

In [ ]:
# Bonus exercise: do exploratory data analysis on our training dataset. E.g. histogram of March temperatures.

Do some machine learning.

In [ ]:
# define a (not-yet-fitted) linear regression model
linreg01 = LinearRegression() # we will use default hyperparameters

# fit the model to our training data
linreg01.fit(X_train, y_train)

# make predictions based on our validation X
y_pred = linreg01.predict(X_val)

# compare predictions to ground truth by calculating a metric
linreg01_rmse = np.sqrt(np.mean((y_val - y_pred)**2)) # RMSE is Root Mean Square Error

In [ ]:
linreg01_rmse

In [ ]:
# define a (not-yet-fitted) random forest model
forest01 = RandomForestRegressor() # we will use default hyperparameters

# fit the model to our training data
forest01.fit(X_train, y_train)

# make predictions based on our validation X
y_pred = forest01.predict(X_val)

# compare predictions to ground truth by calculating a metric
forest01_rmse = np.sqrt(np.mean((y_val - y_pred)**2)) # RMSE is Root Mean Square Error

In [ ]:
forest01_rmse

RMSE describes, on average, how far off the model's predictions of reflectance are from the ground truth reflectance.

The default Random Forest performed better than the default Linear Regression.

Next: try some more models! Try different hyperparameters!

In [ ]:
# Exercise: try an Extra Trees model - class ExtraTreesRegressor() - and see if it performs better.

Note on Scaling: LinearRegression(), RandomForestRegressor(), and ExtraTreesRegressor() do not require scaling of the data. Many other models in scikit-learn do require scaling - your go-to should be StandardScaler(). Scaling your data is just one extra line of code before you fit your model to your data.

Your next science question should be "Is this performance even good?" Relative to other studies? Relative to what is useful?

You should also ask "What else could be done to improve this study?" For example, we would like to control greenness for plant type, as some plants are naturally more green than others.

That's all for today - thanks for participating!

If you have questions outside of the tutorial session, please reach out: posch.au@northeastern.edu and WhatsApp.

-August